In [2]:
!gdown --id 1eSo7XcOuW1BlLLt4YT5Vo-gK1FgR8xMU

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1eSo7XcOuW1BlLLt4YT5Vo-gK1FgR8xMU
From (redirected): https://drive.google.com/uc?id=1eSo7XcOuW1BlLLt4YT5Vo-gK1FgR8xMU&confirm=t&uuid=2bb5fa4c-14a4-4dae-9345-9d27baca2414
To: /kaggle/working/nature_12K.zip
100%|██████████████████████████████████████| 3.82G/3.82G [00:41<00:00, 92.5MB/s]


In [3]:
!unzip -q nature_12K.zip

In [ ]:
https://drive.google.com/file/d/1eSo7XcOuW1BlLLt4YT5Vo-gK1FgR8xMU/view?usp=sharing

In [47]:
from torch.nn import Module, ReLU, Conv2d, Linear, MaxPool2d, LogSoftmax, NLLLoss, Dropout, BatchNorm2d, LeakyReLU, GELU, SELU, Mish
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch import flatten, float, no_grad
from torch.optim import Adam
import torch
import wandb
import math
import matplotlib.pyplot as plt
import numpy as np

210
211
212
213


In [48]:
PARAMETERS = {
    'data_augmentation': True,
    'batch_normalization': False,
    'filters': 64, # no. of filters in first layer
    'filter_org': 'half', # 'half', 'double'
    'dropout': 0.2,
    'activation': 'relu',
    'train_data_dir': "/kaggle/working/inaturalist_12K/train",
    'test_data_dir': "/kaggle/working/inaturalist_12K/val",
    'batch_size': 64,
    'learning_rate': 0.001,
    'epochs': 25,
    'dim': 256,
    'conv_kernel_size': 3,
    'dense_neurons': 1000
}

214
215
216
217


In [49]:
CLASSES = {
    0:'Amphibia',
    1:'Animalia',
    2:'Arachnida',
    3:'Aves',
    4:'Fungi',
    5:'Insecta',
    6:'Mammalia',
    7:'Mollusca',
    8:'Plantae',
    9:'Reptilia'
}

218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241


In [51]:
def get_data(param, type):
    if(type.lower() == 'train'):
        if param['data_augmentation']:
            transform = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.3),
                transforms.RandomRotation(degrees=12),
                transforms.Resize((param['dim'],param['dim'])),
                transforms.ToTensor(), 
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
            ])
        else:
            transform = transforms.Compose([
                transforms.Resize((param['dim'],param['dim'])),
                transforms.ToTensor(), 
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
            ])

        tdataset = datasets.ImageFolder(root=param['train_data_dir'], transform=transform)
        total = len(tdataset)
        train_sample = math.ceil(total*(0.8))
        val_sample = total-train_sample
        # print(total, train_sample, val_sample)
        train_dataset, validation_dataset = torch.utils.data.random_split(tdataset, [train_sample, val_sample])
        train_dataloader = DataLoader(train_dataset, batch_size=param['batch_size'], shuffle=True)
        validation_dataloader = DataLoader(validation_dataset, batch_size=param['batch_size'], shuffle=False)
        return train_dataloader, validation_dataloader
    
    else:
        transform = transforms.Compose([
            transforms.Resize((param['dim'],param['dim'])),
            transforms.ToTensor(), 
            transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
        ])
        test_dataset = datasets.ImageFolder(root=param['test_data_dir'], transform=transform)
        test_dataloader = DataLoader(test_dataset, batch_size=param['batch_size'])
        return test_dataloader

    

In [52]:
class CNN(Module):
    def __init__(self, param):
        super(CNN, self).__init__()
        self.param=param
        self.data_augmentation = param['data_augmentation']
        self.dropout = param['dropout']
        self.act = self.getActivation(param['activation'])
        self.filters = self.filter_logic(param['filters'], param['filter_org'])
        self.conv_ks = param['conv_kernel_size']
        self.dim = param['dim']
        self.bn = param['batch_normalization']
        self.dense_neurons = param['dense_neurons']


        ####### Layer 1 #######
        curr_dim = self.dim
        self.conv1 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=3, out_channels=self.filters[0])
        curr_dim -= (self.conv_ks-1)
        self.act1 = self.act
        if(self.bn): self.bn1 = BatchNorm2d(self.filters[0])
        self.pool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout1 = Dropout(p=self.dropout)

        ####### Layer 2 #######
        self.conv2 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[0], out_channels=self.filters[1])
        curr_dim -= (self.conv_ks-1)
        self.act2 = self.act
        if(self.bn): self.bn2 = BatchNorm2d(self.filters[1])
        self.pool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout2 = Dropout(p=self.dropout)

        ####### Layer 3 #######
        self.conv3 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[1], out_channels=self.filters[2])
        curr_dim -= (self.conv_ks-1)
        self.act3 = self.act
        if(self.bn): self.bn3 = BatchNorm2d(self.filters[2])
        self.pool3 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout3 = Dropout(p=self.dropout)

        ####### Layer 4 #######
        self.conv4 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[2], out_channels=self.filters[3])
        curr_dim -= (self.conv_ks-1)
        self.act4 = self.act
        if(self.bn): self.bn4 = BatchNorm2d(self.filters[3])
        self.pool4 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout4 = Dropout(p=self.dropout)


        ####### Layer 5 #######
        self.conv5 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[3], out_channels=self.filters[4])
        curr_dim -= (self.conv_ks-1)
        self.act5 = self.act
        if(self.bn): self.bn5 = BatchNorm2d(self.filters[4])
        self.pool5 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout5 = Dropout(p=self.dropout)

    
        ####### Fully Connected Layer #######
        self.dense_neurons = curr_dim * curr_dim * self.filters[4]
        self.fc1 = Linear(in_features=(curr_dim * curr_dim * self.filters[4]), out_features=self.dense_neurons)  # How to calculate dimension of filters at previous level
        self.act6 = self.act
        self.dropout6 = Dropout(p=0.5)
        

        ####### Output Layer #######
        self.out = Linear(in_features=self.dense_neurons, out_features=10)
        self.act7 = LogSoftmax(dim=1)


    def getActivation(self, act):
        act = act.lower()
        if(act == 'relu'):
            return ReLU()
        elif(act == 'leakyrelu'):
            return LeakyReLU()
        elif(act == 'gelu'):
            return GELU()
        elif(act == 'selu'):
            return SELU()
        elif(act == 'mish'):
            return Mish()
    

    def filter_logic(self, filter, org):
        level = []
        org = org.lower()
        if org == 'same':
            level = [filter for i in range(5)]
        elif org == 'double':
            level = [filter*pow(2,i) for i in range(5)]
        elif org == 'half':
            level = [max(filter//pow(2,i),1) for i in range(5)]
        return level

    

    def forward(self, r):

        r=self.conv1(r)
        r=self.act1(r)
        if(self.bn): r=self.bn1(r)
        r=self.pool1(r)
        r=self.dropout1(r)

        r=self.conv2(r)
        r=self.act2(r)
        if(self.bn): r=self.bn2(r)
        r=self.pool2(r)
        r=self.dropout2(r)

        r=self.conv3(r)
        r=self.act3(r)
        if(self.bn): r=self.bn3(r)
        r=self.pool3(r)
        r=self.dropout3(r)

        r=self.conv4(r)
        r=self.act4(r)
        if(self.bn): r=self.bn4(r)
        r=self.pool4(r)
        r=self.dropout4(r)

        r=self.conv5(r)
        r=self.act5(r)
        if(self.bn): r=self.bn5(r)
        r=self.pool5(r)
        r=self.dropout5(r)

        r=flatten(r,1)
        r=self.fc1(r)
        r=self.act6(r)
        r=self.dropout6(r)
        
        r=self.out(r)
        output=self.act7(r)

        return output
        


In [15]:
# # For personal debug use
# def train(param):
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     model = CNN(param).to(device)
#     optimizer = Adam(model.parameters(), lr=param['learning_rate'])
#     loss_function = NLLLoss()
#     train_data_loader, validation_data_loader = get_data(param, 'train')
    

#     for epo in range(param['epochs']):
#         model.train()
#         totalTrainLoss = 0
#         totalValLoss = 0
#         trainCorrect = 0
#         valCorrect = 0
#         train_counter=0
#         validation_counter=0
#         for (image, label) in train_data_loader:
#             (image, label) = (image.to(device), label.to(device))
#             prediction = model(image)
#             loss = loss_function(prediction, label)
            
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
            
#             totalTrainLoss += loss
#             trainCorrect += (prediction.argmax(1) == label).type(float).sum().item()
#             train_counter+=1
#             # print(train_counter)
        
#         with no_grad():
#             model.eval()
#             for (image, label) in validation_data_loader:
#                 (image, label) = (image.to(device), label.to(device))
#                 pred = model(image)
#                 totalValLoss += loss_function(pred, label)
#                 valCorrect += (pred.argmax(1) == label).type(float).sum().item()
#                 validation_counter+=1

#         print(f"Epochs --> {epo}")
#         print(f"Train Loss --> {(totalTrainLoss/train_counter).cpu().detach().numpy()}")
#         print(f"Train Accuracy --> {trainCorrect/len(train_data_loader.dataset)}")
#         print(f"Validation Loss --> {(totalValLoss/validation_counter).cpu().detach().numpy()}")
#         print(f"Validation Accuracy --> {valCorrect/len(validation_data_loader.dataset)}")
#         print("##########################################################################################")
    
#     return model

In [16]:
mdl = train(PARAMETERS)

Epochs --> 0
Train Loss --> 2.278223991394043
Train Accuracy --> 0.1295
Validation Loss --> 2.2322638034820557
Validation Accuracy --> 0.16858429214607304
##########################################################################################
Epochs --> 1
Train Loss --> 2.1942501068115234
Train Accuracy --> 0.1955
Validation Loss --> 2.193485736846924
Validation Accuracy --> 0.21410705352676337
##########################################################################################
Epochs --> 2
Train Loss --> 2.1620495319366455
Train Accuracy --> 0.21425
Validation Loss --> 2.139416456222534
Validation Accuracy --> 0.21060530265132565
##########################################################################################
Epochs --> 3
Train Loss --> 2.129056453704834
Train Accuracy --> 0.228125
Validation Loss --> 2.1208577156066895
Validation Accuracy --> 0.23861930965482742
##########################################################################################
Epochs --> 4


KeyboardInterrupt: 

In [ ]:
# test_data_loader = get_data(PARAMETERS, 'test')
# tstCorrect = 0
# tstCounter = 0
# y = []
# y_pred = []
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# with no_grad():
#     mdl.eval()
#     for (image, label) in test_data_loader:
#         (image, label) = (image.to(device), label.to(device))
#         pred = mdl(image)
#         y.extend(label.tolist())
#         y_pred.extend(pred.argmax(1).tolist())
#         # print(pred)
#         tstCorrect += (pred.argmax(1) == label).type(float).sum().item()
#         tstCounter+=PARAMETERS['batch_size']

# print(tstCorrect)
# print(tstCounter)
# print((tstCorrect/tstCounter)*100)

In [53]:
## for wandb sweeps
def train():
    wandb.init()
    param = wandb.config
    wandb.run.name = f'fltr_{param.filters}_fltrOrg_{param.filter_org}_dataAug_{param.data_augmentation}_batchNorm_{param.batch_normalization}_act_{param.activation}_batchSz_{param.batch_size}'

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = CNN(param).to(device)
    optimizer = Adam(model.parameters(), lr=param['learning_rate'])
    loss_function = NLLLoss()
    train_data_loader, validation_data_loader = get_data(param, 'train')
    

    for epo in range(param['epochs']):
        model.train()
        totalTrainLoss = 0
        totalValLoss = 0
        trainCorrect = 0
        valCorrect = 0
        train_counter=0
        validation_counter=0
        for (image, label) in train_data_loader:
            (image, label) = (image.to(device), label.to(device))
            prediction = model(image)
            loss = loss_function(prediction, label)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            totalTrainLoss += loss
            trainCorrect += (prediction.argmax(1) == label).type(float).sum().item()
            train_counter+=1
          
        
        with no_grad():
            model.eval()
            for (image, label) in validation_data_loader:
                (image, label) = (image.to(device), label.to(device))
                pred = model(image)
                totalValLoss += loss_function(pred, label)
                valCorrect += (pred.argmax(1) == label).type(float).sum().item()
                validation_counter+=1

        tr_ls = (totalTrainLoss/train_counter).cpu().detach().numpy()
        tr_acc = trainCorrect/len(train_data_loader.dataset)
        val_ls = (totalValLoss/validation_counter).cpu().detach().numpy()
        val_acc = valCorrect/len(validation_data_loader.dataset)
        print(f"Epoch --> {epo}")
        print(f"Train Loss --> {tr_ls}")
        print(f"Train Accuracy --> {tr_acc}")
        print(f"Validation Loss --> {val_ls}")
        print(f"Validation Accuracy --> {val_acc}")
        
        lg={
            'epoch': epo+1,
            'tr_accuracy': tr_acc,
            'val_accuracy': val_acc,
            'tr_loss': tr_ls,
            'val_loss': val_ls
        }
        wandb.log(lg)

        # return model

Epoch --> 0
Train Loss --> 2.775897741317749
Train Accuracy --> 0.224375
Validation Loss --> 5.420468807220459
Validation Accuracy --> 0.1330665332666333


In [54]:
sweep_config = {
  "method": "bayes", 
  "name": "Q2 Sweep",
  "metric": {"goal": "maximize", "name": "val_accuracy"},
  "parameters": {
    "data_augmentation":{"values": [True,False]},  # List of boolean values
    "batch_normalization":{"values": [True,False]},  # List of boolean values
    "filters":{"values": [32]},  # List of filter values for first layer
    "filter_org":{"values": ["same","half","double"]},  # List of filter organization options.. , "half", "double"
    "dropout":{"values": [0.2,0.3]},  # Dropout rates
    "activation":{"values": ["relu","leakyrelu","gelu","mish","selu"]},  # Activation functions... , "relu", "leaky_relu"
    "batch_size":{"values": [32]},
    "learning_rate":{"values": [0.001,0.0001]},
    "epochs":{"values": [10]},
    "dim":{"values": [256]},
    "conv_kernel_size":{"values": [3,5]},
    "dense_neurons":{"values": [512]},
    "train_data_dir":{"values": ["/kaggle/working/inaturalist_12K/train"]},
    "test_data_dir":{"values": ["/kaggle/working/inaturalist_12K/val"]}
  }
}


In [55]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [56]:
wandb.login(key="b8d44a4abbab8753e976a6e5ab717fd669ba99a2")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [57]:
# wandb.init()
sweep_id = wandb.sweep(sweep_config, project="DA6401_A2")

Create sweep with ID: dxsvcrdc
Sweep URL: https://wandb.ai/cs24m030-indian-institute-of-technology-madras/DA6401_A2/sweeps/dxsvcrdc


In [58]:
wandb.agent(sweep_id, function=train, count=60)
wandb.finish()

wandb: Agent Starting Run: 771qivyv with config:
wandb: 	activation: gelu
wandb: 	batch_normalization: True
wandb: 	batch_size: 32
wandb: 	conv_kernel_size: 3
wandb: 	data_augmentation: False
wandb: 	dense_neurons: 512
wandb: 	dim: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	filter_org: double
wandb: 	filters: 32
wandb: 	learning_rate: 0.0001
wandb: 	test_data_dir: /kaggle/working/inaturalist_12K/val
wandb: 	train_data_dir: /kaggle/working/inaturalist_12K/train


Epoch --> 0
Train Loss --> 2.7477564811706543
Train Accuracy --> 0.2295
Validation Loss --> 3.480196237564087
Validation Accuracy --> 0.15307653826913456
Epoch --> 1
Train Loss --> 2.0919463634490967
Train Accuracy --> 0.290375
Validation Loss --> 3.2130064964294434
Validation Accuracy --> 0.1570785392696348
Epoch --> 2
Train Loss --> 1.941374659538269
Train Accuracy --> 0.330625
Validation Loss --> 3.5195510387420654
Validation Accuracy --> 0.1875937968984492
Epoch --> 3
Train Loss --> 1.8052644729614258
Train Accuracy --> 0.377
Validation Loss --> 4.784743785858154
Validation Accuracy --> 0.16208104052026012
Epoch --> 4
Train Loss --> 1.662177324295044
Train Accuracy --> 0.432375
Validation Loss --> 4.263131618499756
Validation Accuracy --> 0.16908454227113556
Epoch --> 5
Train Loss --> 1.4502959251403809
Train Accuracy --> 0.5055
Validation Loss --> 3.6374661922454834
Validation Accuracy --> 0.22511255627813906
Epoch --> 6
Train Loss --> 1.167826771736145
Train Accuracy --> 0.607625

epoch,▁▂▃▃▄▅▆▆▇█
tr_accuracy,▁▂▂▃▃▄▅▆▇█
tr_loss,█▆▆▅▅▄▃▃▂▁
val_accuracy,▁▁▃▁▂▅▇▇██
val_loss,▂▁▂█▆▃▂▂▂▃
epoch,10
tr_accuracy,0.9005
tr_loss,0.33736
val_accuracy,0.28064
val_loss,3.76524


wandb: Agent Starting Run: zy9oi296 with config:
wandb: 	activation: relu
wandb: 	batch_normalization: False
wandb: 	batch_size: 32
wandb: 	conv_kernel_size: 5
wandb: 	data_augmentation: True
wandb: 	dense_neurons: 512
wandb: 	dim: 256
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	filter_org: double
wandb: 	filters: 32
wandb: 	learning_rate: 0.0001
wandb: 	test_data_dir: /kaggle/working/inaturalist_12K/val
wandb: 	train_data_dir: /kaggle/working/inaturalist_12K/train


Epoch --> 0
Train Loss --> 2.238924026489258
Train Accuracy --> 0.15625
Validation Loss --> 2.1830878257751465
Validation Accuracy --> 0.20410205102551277
Epoch --> 1
Train Loss --> 2.1226861476898193
Train Accuracy --> 0.228125
Validation Loss --> 2.125871419906616
Validation Accuracy --> 0.23761880940470234
Epoch --> 2
Train Loss --> 2.049535036087036
Train Accuracy --> 0.256625
Validation Loss --> 2.084796905517578
Validation Accuracy --> 0.25662831415707854
Epoch --> 3
Train Loss --> 1.9879803657531738
Train Accuracy --> 0.293375
Validation Loss --> 2.045382499694824
Validation Accuracy --> 0.27813906953476736
Epoch --> 4
Train Loss --> 1.9317914247512817
Train Accuracy --> 0.31575
Validation Loss --> 2.0524044036865234
Validation Accuracy --> 0.2816408204102051
Epoch --> 5
Train Loss --> 1.8764338493347168
Train Accuracy --> 0.33825
Validation Loss --> 1.9871675968170166
Validation Accuracy --> 0.3061530765382691
Epoch --> 6
Train Loss --> 1.8323737382888794
Train Accuracy --> 0.3

...

wandb: Agent Starting Run: gggwen6g with config:
wandb: 	activation: relu
wandb: 	batch_normalization: True
wandb: 	batch_size: 32
wandb: 	conv_kernel_size: 3
wandb: 	data_augmentation: True
wandb: 	dense_neurons: 512
wandb: 	dim: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	filter_org: same
wandb: 	filters: 32
wandb: 	learning_rate: 0.0001
wandb: 	test_data_dir: /kaggle/working/inaturalist_12K/val
wandb: 	train_data_dir: /kaggle/working/inaturalist_12K/train


Epoch --> 0
Train Loss --> 2.2024242877960205
Train Accuracy --> 0.2065
Validation Loss --> 2.271855115890503
Validation Accuracy --> 0.17558779389694848
Epoch --> 1
Train Loss --> 2.04990816116333
Train Accuracy --> 0.277625
Validation Loss --> 2.181060314178467
Validation Accuracy --> 0.22661330665332666
Epoch --> 2
Train Loss --> 1.983994722366333
Train Accuracy --> 0.2985
Validation Loss --> 2.294356107711792
Validation Accuracy --> 0.19909954977488745
Epoch --> 3
Train Loss --> 1.9458541870117188
Train Accuracy --> 0.3135
Validation Loss --> 2.489931583404541
Validation Accuracy --> 0.1730865432716358
Epoch --> 4
Train Loss --> 1.904656171798706
Train Accuracy --> 0.327875
Validation Loss --> 2.5052926540374756
Validation Accuracy --> 0.18209104552276137
Epoch --> 5
Train Loss --> 1.8746659755706787
Train Accuracy --> 0.343625
Validation Loss --> 2.4733574390411377
Validation Accuracy --> 0.18359179589794897
Epoch --> 6
Train Loss --> 1.8510658740997314
Train Accuracy --> 0.348
Va

wandb: Ctrl + C detected. Stopping sweep.


AttributeError: 'ZMQDisplayPublisher' object has no attribute '_orig_publish'

epoch,▁▂▃▃▄▅▆▆▇█
tr_accuracy,▁▄▅▅▆▇▇███
tr_loss,█▅▄▄▃▂▂▂▁▁
val_accuracy,▁█▄▁▂▂▄▅▅▂
val_loss,▃▁▃▇▇▇▆▆▆█
epoch,10
tr_accuracy,0.37038
tr_loss,1.78789
val_accuracy,0.18309
val_loss,2.55013
